In [11]:
import cv2
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.activations import *
from keras.optimizers import Adam,RMSprop,SGD
x = np.load("x.npy")
y = np.load("y.npy")
shuffe_index = np.random.permutation(x.shape[0])
N = x.shape[0]
K = int(0.8*N)#number of train samples
index = np.random.permutation(N)
x_train = x[index[0:K]]
y_train = y[index[0:K]]
x_test = x[index[K:]]
y_test = y[index[K:]]
print(N, K, N - K)
print(x.shape)

304 243 61
(304, 64, 64, 3)


In [12]:
print(x_train)



[[[[ 30  54  52]
   [ 29  53  51]
   [ 27  54  51]
   ...
   [ 17  48  45]
   [ 18  49  46]
   [ 20  51  48]]

  [[ 28  52  50]
   [ 23  47  45]
   [ 19  46  43]
   ...
   [ 13  44  41]
   [ 17  48  45]
   [ 20  51  48]]

  [[ 32  56  54]
   [ 22  46  44]
   [ 14  38  36]
   ...
   [ 12  41  38]
   [ 17  46  43]
   [ 20  49  46]]

  ...

  [[ 99 128 125]
   [ 98 127 124]
   [104 131 128]
   ...
   [ 99 131 126]
   [ 98 131 124]
   [ 97 130 123]]

  [[ 98 130 125]
   [ 98 128 123]
   [102 129 126]
   ...
   [ 97 129 124]
   [ 95 127 122]
   [ 92 126 120]]

  [[ 96 128 123]
   [ 96 128 123]
   [ 99 126 123]
   ...
   [ 94 126 121]
   [ 90 124 118]
   [ 88 122 116]]]


 [[[ 56  96  95]
   [ 58  94  94]
   [ 65  90  92]
   ...
   [ 88 142 135]
   [ 89 142 133]
   [ 90 141 133]]

  [[ 56  96  95]
   [ 57  93  93]
   [ 64  89  91]
   ...
   [ 89 141 134]
   [ 89 140 132]
   [ 90 141 133]]

  [[ 53  95  94]
   [ 54  92  92]
   [ 62  90  91]
   ...
   [ 82 133 125]
   [ 83 132 124]
   [ 84 133

In [13]:
model = Sequential()

model.add(Conv2D(8, kernel_size=(5, 5), strides= (2, 2), padding='same',input_shape=( 64, 64, 3,)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, kernel_size=(3, 3), strides= (1, 1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), strides= (1, 1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(2))
model.add(Activation('sigmoid'))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy',optimizer=optimizer)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 8)         608       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 32, 32, 8)         0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 16)        1168      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 16, 16, 16)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 32)         

In [9]:
print (x_test.shape[0], x_train.shape[0])

(61, 243)


In [10]:
def generator(x, y, batch_size):
    samples = x.shape[0]
    steps = samples/batch_size
    counter =0
    while True:
        batch_x = np.array(x[batch_size*counter: batch_size*(counter+1)]).astype(np.float32)
        batch_x = batch_x/255.0
        batch_y = np.array(y[batch_size*counter: batch_size*(counter+1)]).astype(np.float32)
        counter+=1
        yield batch_x, batch_y
        if counter >= steps:
            counter = 0
# batch_size = 128
batch_size = 32
epochs=50
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("models/sign.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history= model.fit_generator(generator(x_train, y_train, batch_size), epochs=epochs, steps_per_epoch = x_train.shape[0]/batch_size, 
                             validation_data=generator(x_test, y_test, batch_size), validation_steps = x_test.shape[0]/batch_size, callbacks=[checkpoint])

Epoch 1/50
7/7 [==============================] - 1s 146ms/step - loss: 0.6455 - val_loss: 0.5486

Epoch 00001: val_loss improved from inf to 0.54857, saving model to models/sign.h5
Epoch 2/50
7/7 [==============================] - 0s 6ms/step - loss: 0.4481 - val_loss: 0.3420

Epoch 00002: val_loss improved from 0.54857 to 0.34198, saving model to models/sign.h5
Epoch 3/50
7/7 [==============================] - 0s 6ms/step - loss: 0.3329 - val_loss: 0.2812

Epoch 00003: val_loss improved from 0.34198 to 0.28117, saving model to models/sign.h5
Epoch 4/50
7/7 [==============================] - 0s 6ms/step - loss: 0.2987 - val_loss: 0.2324

Epoch 00004: val_loss improved from 0.28117 to 0.23243, saving model to models/sign.h5
Epoch 5/50
7/7 [==============================] - 0s 7ms/step - loss: 0.2490 - val_loss: 0.1904

Epoch 00005: val_loss improved from 0.23243 to 0.19043, saving model to models/sign.h5
Epoch 6/50
7/7 [==============================] - 0s 6ms/step - loss: 0.2381 - val

In [2]:
import cv2
import numpy as np
from keras.models import load_model
import time
model = load_model("models/sign.h5")


Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [13]:
t = time.time()
predict = model.predict(x_test/255.0)
print(time.time()-t)
for i in range(x_test.shape[0]):
    img = x_test[i]
    img = cv2.resize(img, (128, 128))
    print(predict[i])
    if predict[i,1]>0.5:
        if predict[i,0]<0.9:
            cv2.rectangle(img, (0,0), (30, 30), (0, 0, 255), -1)
        else:
            cv2.rectangle(img, (98, 0), (128, 30), (0, 0, 255), -1)
    cv2.imshow("img", img)
    k =cv2.waitKey(0)
    if k == ord('q'):
        break
cv2.destroyAllWindows()

0.0145812034607
[0.99999636 1.        ]
[0.00135627 0.99999976]
[0.998657 1.      ]
[0.9998561  0.99999964]
[0.9999871 1.       ]
[0.9998431  0.99999994]
[0.9999994 1.       ]
[1.2496114e-04 9.9999893e-01]
[0.9999989 1.       ]
[2.682209e-07 1.000000e+00]
[8.6426735e-07 1.0000000e+00]
[0.99999845 1.        ]
[0.99993926 1.        ]
[1.8805265e-05 1.0000000e+00]
[0.99972016 1.        ]
[0.99999905 1.        ]
[0.99999714 1.        ]
[3.0961633e-04 9.9999976e-01]
[0.9999729 1.       ]
[8.0764294e-05 1.0000000e+00]


In [14]:
print (x_test.shape)

(61, 64, 64, 3)


In [45]:
def classify():
    img = cv2.imread('test/cropleft1.png')

    img = cv2.resize(img, (64, 64))
#     print img.shape
#     with graph.as_default():
#         set_session(sess)
#         side, conf = model.predict(img)
    side, conf = model.predict(np.array([img])/255.0)[0]
    

    print side, conf


In [46]:
classify()

5.090714e-06 1.0


In [3]:
import timeit
from datetime import timedelta

# sess = tf.Session()

# graph = tf.get_default_graph()

# set_session(sess)
# my_model = load_model('models/sign_fix.h5')
# my_model = load_model('models/sign_fix.h5')

sign_cascade = cv2.CascadeClassifier('models/cascade2.xml')

crop = np.zeros((64, 64), np.uint8)

def FindSign(img):
    signs = sign_cascade.detectMultiScale(img,
                                          scaleFactor=1.03,
                                          minNeighbors=2,
                                          minSize=(19, 19),
                                          maxSize=(60, 60),
                                          flags=0)
    return signs


def Cascade_sign(img_):
    global crop, sess, graph
    t = 2
    side = None
    img = img_[0:100, ...].copy()
    img2 = img_[0:100, ...].copy()
    signs = FindSign(img2)
    position = None
    for (x, y, w, h) in signs:
        if x > 5 and y > 5 and x + w < 475 and y + h < 95 and 1.0 * h / w > 0.85 and 1.0 * h / w < 1.3:
            print(w, h)
            position = [x, y, w, h]
            crop = img[y - t:y + h + t, x - t:x + w + t, :]
            crop = cv2.resize(crop, (64, 64))
            start = timeit.default_timer()
#             with graph.as_default():
#                 set_session(sess)
#                 _side, confident = my_model.predict(np.array([crop]) /
#                                                     255.0)[0]
#                 print('toi dang o day')
#                 print('confident = ', confident)
            _side, confident = model.predict(np.array([crop]) /
                                                    255.0)[0]
            stop = timeit.default_timer()
            print ('Thoi gian chay model la ', stop - start)
            #_side, confident = sign_model.predict(np.array([crop])/255.0)[0]
            # _side = 0
            # confident = 0.9
            if _side < 0.5:
                _side = 0
            else:
                _side = 1
            if confident > 0.5:
                side = _side
            # cv2.rectangle(img,(x-t,y-t),(x+w+t,y+h+t),(0,255,0),2)
            # font = cv2.FONT_HERSHEY_SIMPLEX
            # cv2.putText(img,str(confident),(x-10,y-10), font, 0.5, (255,255,255), 1, cv2.LINE_AA)
            # #cv2.imshow('crop', crop)
    return crop, side, img, position

# time.sleep(5)


def testImage():
    img = cv2.imread('test/left1.png')

    crop1, side, img2, pos = Cascade_sign(img)
    #cv2.imshow('crop', crop)
    #cv2.imshow('sign', img2)
    #cv2.imshow('image', img)
    print ('side= ', side)
    print ('pos= ', pos)
    #getMask(img)

    cv2.waitKey(0)


In [ ]:
testImage()

In [44]:
img = cv2.imread('test/left1.png')

signs = FindSign(img)

for (x, y, w, h) in signs:
        if x > 5 and y > 5 and x + w < 475 and y + h < 95 and 1.0 * h / w > 0.85 and 1.0 * h / w < 1.3:
            print(w, h)

(26, 26)


In [ ]:
cl